In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./Data/Beijing_Tiantan/PRSA_Data_Tiantan_20130301-20170228.csv")

In [3]:
df = df[['year', 'month', 'day', 'hour', 
         'PM2.5', 'PM10', 'SO2', 'NO2',
         'CO', 'O3', 'TEMP', 'PRES', 'DEWP']]

In [4]:
df.head()

,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP
0,2013,3,1,0,6.0,6.0,4.0,8.0,300.0,81.0,-0.5,1024.5,-21.4
1,2013,3,1,1,6.0,29.0,5.0,9.0,300.0,80.0,-0.7,1025.1,-22.1
2,2013,3,1,2,6.0,6.0,4.0,12.0,300.0,75.0,-1.2,1025.3,-24.6
3,2013,3,1,3,6.0,6.0,4.0,12.0,300.0,74.0,-1.4,1026.2,-25.5
4,2013,3,1,4,5.0,5.0,7.0,15.0,400.0,70.0,-1.9,1027.1,-24.5


In [5]:
df['date'] = df.apply(lambda x: f"{int(x['year'])}-{int(x['month']):02}-{int(x['day']):02} {int(x['hour']):02}", axis=1)
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H')
df = df.drop(["year", "month", "day", "hour"], axis=1)
df = df.set_index('date')
df.index.freq = 'H'

In [6]:
df = df.resample('D').mean().round(2)

In [7]:
def impute(station_df, column, window_length=7):
    try:
        for idx, value in enumerate(df[column].values):
            if np.isnan(value):
                df[column][idx] = df[column][idx - window_length]
    except:
        pass
    return df

In [8]:
for column in df.columns:
    df = impute(df, column, window_length=7)

In [9]:
df.head()

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP
date,,,,,,,,,
2013-03-01,8.62,14.46,8.88,21.58,450.00,68.08,1.33,1028.78,-21.47
2013-03-02,31.71,37.04,27.17,50.48,934.78,34.04,0.49,1028.85,-16.96
2013-03-03,98.04,121.33,55.62,82.96,2028.96,16.92,4.99,1016.46,-12.31
2013-03-04,22.33,37.58,16.58,42.04,895.79,53.12,10.56,1019.45,-14.00
2013-03-05,141.67,156.38,74.33,101.67,2408.00,94.00,7.21,1012.70,-7.78


In [10]:
pm25_cols = ['PM2.5', 'TEMP', 'PRES', 'DEWP']
df[pm25_cols].to_pickle("./Data/beijing_pm25_dataset.pkl", protocol=4)

pm10_cols = ['PM10', 'TEMP', 'PRES', 'DEWP']
df[pm10_cols].to_pickle("./Data/beijing_pm10_dataset.pkl", protocol=4)